# Лабораторная работа №4: Свойства оценок, полученных методом максимального правдоподобия 

**Цели и задачи л/р**: применить на практике метод максимального правдоподобия,
*оценить статистические свойства получаемых с их помощью статистических оценок
*параметров распределений. 

## Задание.
1. Применить метод максимального правдоподобия к равномерному закону
   распределения, чья плотность задана как $\varphi_r(x, a, b) = \begin{cases}\cfrac{1}{b-a}, x\in[a,b]\\0, x\not\in[a,b]\end{cases}$ 
   Пусть выборка значений соответствующей случайной величины $r$ задана как
   $x_1,x_2,x_3...x_n$. Построить оценки $a^*$ и $b^*$ для параметров $a$ и $b$.
2. Применить метод максимального правдоподобия к закону распределения Лапласа,
   чья плотность задана как $\varphi_r(x, c, \lambda)=\cfrac{\lambda}{a}\cdot e^{-\lambda|x-c|}$.
   Пусть выборка значений соответствующей случайной величины $r$ задана как
   $x_1,x_2,x_3...x_n$. Построить оценку $\lambda^*$ для параметра $\lambda$. 
3. Запрограммировать расчеты по полученным оценкам и построить график зависимости 
оценки их среднеквадратического отклонения (см. лаб. раб. 3) от размера выборки.